# Libruaries import


In [1]:
! git clone https://github.com/marcoguerra192/MinScaffold

Cloning into 'MinScaffold'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (231/231), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 231 (delta 34), reused 216 (delta 25), pack-reused 0
Receiving objects: 100% (231/231), 12.16 MiB | 25.16 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [2]:
import sys
sys.path.append('/content/MinScaffold/src/')

import Driver
import Geometry
import Scaffold

import math
import numpy as np
from random import random
import networkx as nx
import pickle as pk
import matplotlib.pyplot as plt


sys.path.append('/content/MinScaffold')
import Holes as ho

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data import and preperation

There we take data from specific part of brain and get scaffold out of it. We prepare data using partial correlation as metrix of distance for homological scaffold.

In [4]:
import pandas as pd
from scipy import stats, linalg

In [5]:
import numpy as np
from scipy import stats, linalg

def partial_corr(C):
    """
    Returns the sample linear partial correlation coefficients between pairs of variables in C, controlling 
    for the remaining variables in C.
    Parameters
    ----------
    C : array-like, shape (n, p)
        Array with the different variables. Each column of C is taken as a variable
    Returns
    -------
    P : array-like, shape (p, p)
        P[i, j] contains the partial correlation of C[:, i] and C[:, j] controlling
        for the remaining variables in C.
    """
    
    C = np.asarray(C)
    p = C.shape[1]
    P_corr = np.zeros((p, p), dtype=np.float)
    for i in range(p):
        P_corr[i, i] = 1
        for j in range(i+1, p):
            idx = np.ones(p, dtype=np.bool)
            idx[i] = False
            idx[j] = False
            beta_i = linalg.lstsq(C[:, idx], C[:, j])[0]
            beta_j = linalg.lstsq(C[:, idx], C[:, i])[0]

            res_j = C[:, j] - C[:, idx].dot( beta_i)
            res_i = C[:, i] - C[:, idx].dot(beta_j)
            
            corr = stats.pearsonr(res_i, res_j)[0]
            P_corr[i, j] = corr
            P_corr[j, i] = corr
        
    return P_corr

In [6]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sub-01.csv')
columns = list(data.columns)
Frontal_1 = columns[2:16]
Frontal_2 = columns[22:26]
Lingual = columns[46:48]
Occipital = columns[48:54]
Parietal = columns[58:62]
Temporal = columns[80:90]
Cerebelum = columns[91:109]

In [8]:
Frontal = []
for i in Frontal_1:
  Frontal.append(i)

for i in Frontal_2:
  Frontal.append(i)

In [9]:
healthy = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '51', '52', '54', '55', '56']

Scaffolds = []
SS = []
c = 0
for pat in healthy:
    print(c)
    data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sub-'+ pat +'.csv')
    df_cerebelum = data[Cerebelum]
    df_cerebelum_corr = partial_corr(df_cerebelum)
    W = abs(1 - df_cerebelum_corr).round(4)
    epsList = Geometry.genFullEpsList(W)
    F = Driver.getFiltrBasis(W, epsList, Draws=False, parallel=True, monitor=None)
    S = Scaffold.frequency_Scaffold(F , len(W))
    Scaf = nx.from_numpy_matrix(S, parallel_edges=False)
    SS.append(S)
    Scaffolds.append(Scaf)
    c+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [10]:
degrees = []
for scaf in Scaffolds:
   degrees.append(list(scaf.degree()))

strength = []
for i in range(len(SS)):
  strength.append(SS[i].sum(0))

In [11]:
BC = []
for scaf in Scaffolds:
  betw_centrality = nx.algorithms.centrality.betweenness_centrality(scaf)
  BC.append(betw_centrality)

df_cerebelum_bc = pd.DataFrame(BC)
df_cerebelum_bc = df_cerebelum_bc.transpose()

CC = []
for scaf in Scaffolds:
  betw_centrality = nx.algorithms.centrality.closeness_centrality(scaf)
  CC.append(betw_centrality)

df_cerebelum_cc = pd.DataFrame(CC)
df_cerebelum_cc = df_cerebelum_cc.transpose()

In [12]:
for i in range(18):
  df_cerebelum_bc = df_cerebelum_bc.rename(index={i: Cerebelum[i]})

for i in range(18):
  df_cerebelum_cc = df_cerebelum_cc.rename(index={i: Cerebelum[i]})

In [13]:
y_list = []
for n in range(23):
  y = []
  for i, j in degrees[n]:
    y.append(j)
  y_list.append(y)


df_cerebelum_degrees = pd.DataFrame(index=Cerebelum)
for i in range(23):
    df_cerebelum_degrees[i] = pd.DataFrame(y_list[i], index=Cerebelum)

df_cerebelum_strength = pd.DataFrame(index=Cerebelum)
for i in range(23):
    df_cerebelum_strength[i] = pd.DataFrame(strength[i], index=Cerebelum)

In [14]:
df_cerebelum_strength['dislexic'] = df_cerebelum_strength.iloc[:, 0:18].mean(axis=1)
df_cerebelum_strength['healthy'] = df_cerebelum_strength.iloc[:, 18:].mean(axis=1)
df_cerebelum_bc['dislexic'] = df_cerebelum_bc.iloc[:, 0:18].mean(axis=1)
df_cerebelum_bc['healthy'] = df_cerebelum_bc.iloc[:, 18:].mean(axis=1)
df_cerebelum_cc['dislexic'] = df_cerebelum_cc.iloc[:, 0:18].mean(axis=1)
df_cerebelum_cc['healthy'] = df_cerebelum_cc.iloc[:, 18:].mean(axis=1)
df_cerebelum_cc

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,dislexic,healthy
Cerebelum_Crus1_L,0.653846,0.548387,0.629630,0.653846,0.586207,0.653846,0.566667,0.566667,0.629630,0.566667,0.607143,0.607143,0.531250,0.586207,0.680000,0.607143,0.607143,0.653846,0.548387,0.607143,0.515152,0.629630,0.473389,0.607515,0.563536
Cerebelum_Crus1_R,0.607143,0.566667,0.566667,0.531250,0.607143,0.629630,0.607143,0.607143,0.629630,0.531250,0.548387,0.607143,0.586207,0.586207,0.586207,0.566667,0.607143,0.586207,0.629630,0.607143,0.586207,0.548387,0.473389,0.586546,0.571884
Cerebelum_Crus2_L,0.607143,0.566667,0.607143,0.607143,0.548387,0.607143,0.607143,0.680000,0.548387,0.607143,0.500000,0.629630,0.653846,0.515152,0.548387,0.629630,0.566667,0.653846,0.566667,0.566667,0.653846,0.531250,0.432225,0.593525,0.557363
Cerebelum_Crus2_R,0.586207,0.629630,0.515152,0.607143,0.629630,0.653846,0.653846,0.607143,0.586207,0.629630,0.515152,0.607143,0.459459,0.629630,0.653846,0.607143,0.566667,0.607143,0.607143,0.629630,0.548387,0.586207,0.331373,0.596923,0.549944
Cerebelum_3_L,0.586207,0.629630,0.680000,0.472222,0.548387,0.607143,0.566667,0.629630,0.653846,0.586207,0.586207,0.566667,0.531250,0.653846,0.653846,0.531250,0.531250,0.629630,0.586207,0.531250,0.586207,0.515152,0.451872,0.591327,0.543669
Cerebelum_3_R,0.586207,0.566667,0.607143,0.566667,0.607143,0.586207,0.566667,0.607143,0.548387,0.607143,0.485714,0.607143,0.653846,0.586207,0.629630,0.607143,0.607143,0.607143,0.629630,0.586207,0.607143,0.680000,0.432225,0.590741,0.587658
Cerebelum_4_5_L,0.680000,0.607143,0.531250,0.566667,0.607143,0.629630,0.653846,0.548387,0.607143,0.607143,0.607143,0.607143,0.629630,0.607143,0.629630,0.653846,0.586207,0.607143,0.548387,0.629630,0.515152,0.548387,0.451872,0.609235,0.550444
Cerebelum_4_5_R,0.607143,0.629630,0.586207,0.629630,0.607143,0.607143,0.629630,0.629630,0.500000,0.607143,0.586207,0.653846,0.515152,0.629630,0.586207,0.586207,0.586207,0.629630,0.607143,0.500000,0.586207,0.531250,0.368192,0.600355,0.532191
Cerebelum_6_L,0.607143,0.607143,0.653846,0.607143,0.607143,0.629630,0.586207,0.653846,0.607143,0.607143,0.515152,0.607143,0.586207,0.566667,0.548387,0.629630,0.566667,0.607143,0.653846,0.485714,0.607143,0.515152,0.473389,0.599632,0.555813
Cerebelum_6_R,0.485714,0.586207,0.566667,0.653846,0.629630,0.629630,0.586207,0.653846,0.586207,0.566667,0.629630,0.500000,0.607143,0.586207,0.607143,0.607143,0.653846,0.607143,0.472222,0.607143,0.680000,0.566667,0.451872,0.596826,0.562455


In [15]:
df_cerebelum_degrees['healthy'] = df_cerebelum_degrees.iloc[:, 0:18].mean(axis=1)
df_cerebelum_degrees['dislexic'] = df_cerebelum_degrees.iloc[:, 18:].mean(axis=1)

In [16]:
df_new = df_cerebelum_degrees['healthy']
df_new = df_new.reset_index()
df_new_stre = df_cerebelum_strength['healthy']
df_new_stre = df_new_stre.reset_index()
df_new_bc = df_cerebelum_bc['healthy']
df_new_bc = df_new_bc.reset_index()
df_new_cc = df_cerebelum_cc['healthy']
df_new_cc = df_new_cc.reset_index()

In [17]:
df_new2 = df_cerebelum_degrees['dislexic']
df_new2 = df_new2.reset_index()
df_new2_stre = df_cerebelum_strength['dislexic']
df_new2_stre = df_new2_stre.reset_index()
df_new2_bc = df_cerebelum_bc['dislexic']
df_new2_bc = df_new2_bc.reset_index()
df_new2_cc = df_cerebelum_cc['dislexic']
df_new2_cc = df_new2_cc.reset_index()

In [18]:
df_new = df_new.rename(columns={'healthy': 'Mean'})
df_new2 = df_new2.rename(columns={'dislexic': 'Mean'})
df_new_stre = df_new_stre.rename(columns={'healthy': 'Mean'})
df_new2_stre = df_new2_stre.rename(columns={'dislexic': 'Mean'})
df_new_bc = df_new_bc.rename(columns={'healthy': 'Mean'})
df_new2_bc = df_new2_bc.rename(columns={'dislexic': 'Mean'})
df_new_cc = df_new_cc.rename(columns={'healthy': 'Mean'})
df_new2_cc = df_new2_cc.rename(columns={'dislexic': 'Mean'})

In [19]:
df_new['Condition'] = 'healthy'
df_new2['Condition'] = 'dislexic'
df_new_stre['Condition'] = 'healthy'
df_new2_stre['Condition'] = 'dislexic'
df_new_bc['Condition'] = 'healthy'
df_new2_bc['Condition'] = 'dislexic'
df_new_cc['Condition'] = 'healthy'
df_new2_cc['Condition'] = 'dislexic'

In [20]:
df_mean_degree = pd.concat([df_new, df_new2], ignore_index=True)
df_mean_strength = pd.concat([df_new_stre, df_new2_stre], ignore_index=True)
df_mean_bc = pd.concat([df_new_bc, df_new2_bc], ignore_index=True)
df_mean_cc = pd.concat([df_new_cc, df_new2_cc], ignore_index=True)

In [21]:
df_mean_degree = df_mean_degree.sort_values(by='Mean', ascending=False)
df_mean_strength = df_mean_strength.sort_values(by='Mean', ascending=False)
df_mean_bc = df_mean_bc.sort_values(by='Mean', ascending=False)
df_mean_cc = df_mean_cc.sort_values(by='Mean', ascending=False)
df_mean_cc

,index,Mean,Condition
32,Cerebelum_9_L,0.614897,dislexic
31,Cerebelum_8_R,0.610550,dislexic
24,Cerebelum_4_5_L,0.609235,dislexic
18,Cerebelum_Crus1_L,0.607515,dislexic
25,Cerebelum_4_5_R,0.600355,dislexic
26,Cerebelum_6_L,0.599632,dislexic
35,Cerebelum_10_R,0.599225,dislexic
28,Cerebelum_7b_L,0.599049,dislexic
21,Cerebelum_Crus2_R,0.596923,dislexic
27,Cerebelum_6_R,0.596826,dislexic


# Visualization

In [22]:
import plotly.express as px

In [23]:
fig = px.bar(df_mean_degree, x='index', y='Mean', color='Condition', barmode='group', log_y=False)
fig.show()

In [24]:
fig = px.bar(df_mean_strength, x='index', y='Mean', color='Condition', barmode='group', log_y=False)
fig.show()

In [25]:
fig = px.bar(df_mean_bc, x='index', y='Mean', color='Condition', barmode='group', log_y=False)
fig.show()

In [26]:
fig = px.bar(df_mean_cc, x='index', y='Mean', color='Condition', barmode='group', log_y=True)
fig.show()

# Testing

In [27]:
from scipy.stats import ks_2samp
from scipy.stats import wasserstein_distance

#perform Kolmogorov-Smirnov test on degree sequence
ks_2samp(df_cerebelum_degrees['healthy'], df_cerebelum_degrees['dislexic'])

Ks_2sampResult(statistic=0.6111111111111112, pvalue=0.0018396816629279345)

In [28]:
#perform Kolmogorov-Smirnov test on strength
ks_2samp(df_cerebelum_strength['healthy'], df_cerebelum_strength['dislexic'])

Ks_2sampResult(statistic=0.6111111111111112, pvalue=0.0018396816629279345)

In [29]:
ks_2samp(df_cerebelum_bc['healthy'], df_cerebelum_bc['dislexic'])

Ks_2sampResult(statistic=0.2222222222222222, pvalue=0.7810478607409853)

In [30]:
ks_2samp(df_cerebelum_cc['healthy'], df_cerebelum_cc['dislexic'])

Ks_2sampResult(statistic=0.9444444444444444, pvalue=7.933766012281933e-09)